## Notebook for CNNs

### Part 1

#### (a)
As the assignment specifies that we must use vanilla python, I implemented the feature without numpy, I also created a printConvolved function so that it would be easier to read the output. I also assumed that the stride would always be 1, so as to simplify the question since it was never specified.

To convolve a matrix, I simply loop through all the points that could be multiplied by the top left corner element in the kernel, I then get the local matrix, a sub matrix that contains the points that will be multiplied by the kernel for this iteration, and since this is done in vanilla python I have to multiply the matrix out, the flatten it to a 1d list, then sum the values inside to get an element in the convolved array.

In [74]:
def convolve(n, k):
    outputLen = len(n) - len(k) + 1

    convMatrix = []
    localMatrixSizeModifier = len(k)
    for row in range(outputLen):
        convRow = []
        for col in range(outputLen):
            localMatrix = [i[col : col + localMatrixSizeModifier] for i in n[row : row + localMatrixSizeModifier]]
            multMatrices = [[x * y for x, y in zip(a, b)] for a, b in zip(localMatrix, k)]
            flattenedMatrices = sum(multMatrices, [])
            convRow.append(sum(flattenedMatrices))
        convMatrix.append(convRow)
    return convMatrix

The print function will simply add walls to the ends of each row, and will add the correct number of spaces for each each input so that they will all sit at a uniform starting place for the first number, the largest number is considered to be in the thousands, and a space before any number if it is not negative.

In [4]:
import math

def printConvolved(c):
    for row in c:
        toPrint = '|'
        for elem in row:
            if elem != 0:
                newElem = ''
                logged = math.log10(abs(elem))
                if logged % 1 != 0:
                    spaces = 5 - (math.ceil(logged))
                else:
                    spaces = 4 - (math.ceil(logged))
                newElem = str(elem) + (' ' * spaces)
                if elem > 0:
                    newElem = ' ' + newElem
                toPrint += newElem
            else:
                toPrint += ' ' + str(elem) + (' ' * 4)
        print(toPrint + '|')

In [75]:
n = [[1, 2, 3, 4, 5], 
    [1, 3, 2, 3, 10], 
    [3, 2, 1, 4, 5], 
    [6, 1, 1, 2, 2], 
    [3, 2, 1, 5, 4]]
k = [[1, 0, -1], 
    [1, 0, -1], 
    [1, 0, -1]]

printConvolved(convolve(n, k))

|-1    -4    -14   |
| 6    -3    -13   |
| 9    -6    -8    |


#### (b)
The image I used for this was a black square that was placed on a 40x40 white page, the square does not take up the whole page, I chose this size because on my 13" screen, when convolved and printed with the function I made, it fits perfectly width wise in a terminal, making it easy to read and understand. 

In [76]:
import numpy as np
import PIL.Image as Image

im = Image.open('square.jpg')
rgb = np.array(im.convert('RGB'))
r = rgb[:,:,0]

kernel1 = [[-1, -1, -1], 
           [-1, 8, -1], 
           [-1, -1, -1]]
kernel2 = [[0, -1, 0], 
           [-1, 8, -1], 
           [0, -1, 0]]
con1 = convolve(r, kernel1)
printConvolved(con1)
print("\n\n")
con2 = convolve(r, kernel2)
printConvolved(con2)

| 0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0    |
| 0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0    |
| 0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0    |
| 0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0    |
| 0     0     0     0     0     0     0     0     0     0     0     0     0 

### Part 2

#### (a)
The code here creates a convolutional neural network where each hidden layer uses relu for the activation function, 3x3 kernels and same padding. The first layer makes 16 output channels, the second layer has 16 output channels and uses a stride of 2, the third layer outputs 32 channels, and the 4th hidden layer outputs 32 channels and uses a stride of 2 again. We use a dropout regularizer of 50% and then use a soft max dense layer with an l1 regularizer with c = 0.0001.

It uses batch sizes of 128 and will train the model over 20 epochs.

In [ ]:
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, LeakyReLU, MaxPooling2D
from sklearn.metrics import confusion_matrix, classification_report


# Model / data parameters
num_classes = 10
input_shape = (32, 32, 3)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
n=5000

# takes the first 5000 datapoints in x_train and y_train
x_train = x_train[1:n]; y_train=y_train[1:n]
#x_test=x_test[1:500]; y_test=y_test[1:500]

# images are originaly in 0-255 values, regularize them to 0-1
# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
print("orig x_train shape:", x_train.shape)

# data currently saved as a vector with numbers 0-9, instead, turn each y into a list of binary numbers, with 9 elements in each list
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

use_saved_model = False
if use_saved_model:
    model = keras.models.load_model("cifar.model")
else:
    model = keras.Sequential()
    model.add(Conv2D(16, (3,3), padding='same', input_shape=x_train.shape[1:],activation='relu'))
    model.add(Conv2D(16, (3,3), strides=(2,2), padding='same', activation='relu'))
    model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(32, (3,3), strides=(2,2), padding='same', activation='relu'))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax',kernel_regularizer=regularizers.l1(0.0001)))
    model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])
    model.summary()

    batch_size = 128
    epochs = 20
    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
    model.save("cifar.model")
    plt.subplot(211)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.subplot(212)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss'); plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

preds = model.predict(x_train)
y_pred = np.argmax(preds, axis=1)
y_train1 = np.argmax(y_train, axis=1)
print(classification_report(y_train1, y_pred))
print(confusion_matrix(y_train1,y_pred))

preds = model.predict(x_test)
y_pred = np.argmax(preds, axis=1)
y_test1 = np.argmax(y_test, axis=1)
print(classification_report(y_test1, y_pred))
print(confusion_matrix(y_test1,y_pred))